In [1]:
import argparse, os, time
import pandas as pd
import geopandas as gpd
import polars as pl
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import ScalarFormatter, FuncFormatter
import matplotlib.patheffects as path_effects
from matplotlib.cm import get_cmap
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from collections import defaultdict
import warnings
from tqdm import tqdm
import pickle
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd


tqdm.pandas()
warnings.filterwarnings("ignore")

In [2]:
year = 20
pred_csv = f"./output/donors_state{year}_pred_lastname.csv"
donors_csv = f"./data/CampaignFin{year}/donors_recip{year}.csv"
cands_csv = f"./data/CampaignFin{year}/cands{year}.txt"
cmtes_csv = f"./data/CampaignFin{year}/cmtes{year}.txt"

In [3]:
donors = pd.read_csv(donors_csv)
donors.head(10)

,contrib_id,recip_id,name,name_new,realcode,gender,occupation,employer,city,state,total_donated,donation_count,avg_donation,med_donation
0,U00000037041,N00029349,"BLOOMBERG, MICHAEL R",michael r bloomberg,Z9000,M,FOUNDER,BLOOMBERG INC.,NEW YORK,NY,1.086565e+09,868,1.251802e+06,612.0
1,U00000036521,N00044966,"STEYER, TOM",tom steyer,Z9000,M,PRESIDENTIAL CANDIDATE,SELF-EMPLOYED,SAN FRANCISCO,CA,3.179460e+08,582,5.462989e+05,2036.5
2,,N00001669,"INFORMATION REQUESTED, INFORMATION",information information requested,Y4000,,NaN,NaN,NEW YORK,CA,9.947018e+07,589,1.688798e+05,250.0
3,,C00484642,MAJORITY FORWARD,majority forward majority forward,Z9600,,NaN,NaN,WASHINGTON,DC,9.866908e+07,122,8.087629e+05,117035.5
4,,C00618371,WINRED,winred winred,Z9600,,NaN,NaN,ARLINGTON,VA,9.195982e+07,197,4.668011e+05,500.0
5,,C00571703,CHEVRON CORPORATION,koch industries inc koch industries inc,Y4000,,NaN,NaN,WASHINGTON,TX,6.460129e+07,212,3.047231e+05,50000.0
6,,C00495028,CHC BOLD PAC,chc bold pac chc bold pac,Z9600,,NaN,NaN,WASHINGTON,DC,5.408387e+07,207,2.612747e+05,70000.0
7,,C00693382,FUND FOR POLICY REFORM,fund for policy reform fund for policy reform,J1200,,NaN,NaN,WILMINGTON,DE,4.950000e+07,7,7.071429e+06,6000000.0
8,,C00669259,FUTURE FORWARD USA ACTION,future forward usa action future forward usa a...,J1200,,NaN,NaN,WASHINGTON,DC,4.526326e+07,27,1.676417e+06,100000.0
9,,C00504530,AMERICAN ACTION NETWORK,american action network american action network,Z9600,,NaN,NaN,WASHINGTON,DC,4.465169e+07,138,3.235630e+05,100000.0


In [4]:
cands_lf = (
        pl.scan_csv(
            cands_csv,
            separator=',', 
            quote_char='|', 
            encoding='utf8-lossy', 
            has_header=False,
            new_columns=['dummy1', 'id', 'recip_id', 'name', 'party', 
                        'seat', 'seat_current', 'ran_general', 'ran_ever', 'type', 
                        'recipcode', 'nopacs'],
            ignore_errors=True
        )
        .select(['id', 'recip_id', 'name', 'party', 'seat', 'seat_current', 
                 'ran_general', 'ran_ever', 'type', 'recipcode'])
    )

cands = cands_lf.collect()
cands = cands.to_pandas()
cands = cands.drop_duplicates(subset=['recip_id', 'name', 'party', 'seat'], keep='last')
cands[(cands["seat"] == "PRES") & (cands["ran_general"] == "Y")]

,id,recip_id,name,party,seat,seat_current,ran_general,ran_ever,type,recipcode
6211,P00013524,N00011042,Jo Jorgensen (L),L,PRES,,Y,Y,C,3L
6278,P00014209,N00042370,Don Blankenship (3),3,PRES,,Y,Y,C,3L
6934,P80000722,N00001669,Joe Biden (D),D,PRES,,Y,Y,C,DW
6936,P80001571,N00023864,Donald Trump (R),R,PRES,PRES,Y,Y,I,RL


In [5]:
lf = (
        pl.scan_csv(
            cmtes_csv,
            separator=',', 
            quote_char='|', 
            encoding='utf8-lossy', 
            has_header=False,
            new_columns=['dummy1', 'cmte_id', 'pac_short', 'affiliate', 'pac', 
                        'recip_id', 'recipcode', 'cand_id', 'party', 'prim_code', 
                        'source', 'sensitive', 'foreign', 'active'],
            ignore_errors=True
        )
        .select(['cmte_id', 'pac_short', 'affiliate', 'pac',
                 'recip_id', 'recipcode', 'cand_id', 'party', 'prim_code',
                 'source', 'sensitive', 'foreign', 'active'])
    )
cmtes = lf.collect(streaming=True)
cmtes = cmtes.to_pandas()
cmtes[cmtes["party"] == "L"]

,cmte_id,pac_short,affiliate,pac,recip_id,recipcode,cand_id,party,prim_code,source,sensitive,foreign,active
1426,C00255695,Libertarian National Cmte,Libertarian Party,Libertarian Party,C00255695,3P,None,L,Z5300,Rept,n,0,1
1745,C00315713,Libertarian Party of Illinois,Libertarian Party,Libertarian Party,C00315713,3P,None,L,Z5300,Rept,N,0,1
1878,C00332221,Libertarian Assn of Massachusetts,Libertarian Party,Libertarian Party,C00332221,3P,None,L,Z5300,Rept,N,0,1
2649,C00399865,Libertarian Party of Virginia,Libertarian Party,Libertarian Party,C00399865,3P,None,L,Z5300,Rept,N,0,1
2658,C00400945,Libertarian Party of South Carolina,Libertarian Party,Libertarian Party,C00400945,3P,None,L,Z5300,Rept,n,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16745,C00758250,Liz Terwilliger for Congress,None,Liz Terwilliger for Congress,N00047569,3L,H0PA12207,L,Z1300,Rept,N,0,1
16861,C00759423,COMMITTEE TO ELECT TWHAMBY ...,None,None,C00759423,None,None,L,None,None,None,0,0
17029,C00761130,PRESTON NELSON FOR CONGRESS ...,None,None,N00042426,3L,H0IL08165,L,None,None,N,0,0
17181,C00762732,ITSBEAUTIME ...,None,None,N00047809,3N,P40004277,L,None,None,N,0,0


In [6]:
mapping = {
    'A': 'Agribusiness',
    'B': 'Construction',
    'C': 'Tech',
    'D': 'Defense',
    'E': 'Energy',
    'F': 'Finance',
    'M': 'Misc Business',
    'H': 'Health',
    'J': 'Ideology/Single Issue',
    'K': 'Law',
    'L': 'Labor',
    'M': 'Manufacturing',
    'T': 'Transportation',
    'W': 'Other',
    'Y': 'Unknown',
    'Z': 'Party'
}

donors["sector"] = donors["realcode"].apply(lambda x: str(x).upper()[0]).map(mapping)
donors["sector"].value_counts()

sector
Party                    1299587
Unknown                  1036328
Ideology/Single Issue     861610
Health                    541522
Finance                   443132
Tech                      224719
Law                       199207
Construction              122016
Transportation            118605
Labor                     112626
Energy                     89268
Agribusiness               73594
Manufacturing              55259
Defense                    37952
Name: count, dtype: int64

In [7]:
preds = pd.read_csv(pred_csv)
donors = donors.merge(preds[['contrib_id', 'ethnic']], on='contrib_id', how='left')
donors = donors.merge(cands, on='recip_id', how='left')

In [8]:
donors = donors.merge(cmtes[
    ["cmte_id", "pac_short", "affiliate", "pac", "recip_id", "recipcode", "cand_id", "party"]], 
    left_on='recip_id', right_on="cmte_id", how='left', suffixes=[None, "_pac"]
    ).drop_duplicates(subset=['contrib_id', 'recip_id', 'cmte_id'], keep='last')

donors.loc[donors["pac"].notna(), "name_y"] = donors.loc[donors["pac"].notna(), "pac"]
donors.loc[donors["pac_short"].notna(), "name_y"] = donors.loc[donors["pac_short"].notna(), "pac_short"]

donors["recip_is_pac"] = False
donors.loc[donors["pac_short"].notna(), "recip_is_pac"] = True

donors.loc[donors["pac_short"].notna(), "party"] = donors.loc[donors["pac_short"].notna(), "party_pac"]

In [9]:
donors.loc[donors["name_y"] == "Stop Republicans PAC", "party"] = "D"
donors.loc[donors["name_y"] == "Senate Majority PAC", "party"] = "D"
donors.loc[donors["name_y"] == "314 Action Fund", "party"] = "D"
donors.loc[donors["name_y"] == "Progressive Takeover", "party"] = "D"
donors.loc[donors["name_y"] == "Black Americans to Re-Elect the President", "party"] = "R"
donors.loc[donors["name_y"] == "National Victory Action Fund", "party"] = "R"
donors.loc[donors["name_y"] == "MeidasTouch", "party"] = "D"
donors.loc[donors["name_y"] == "Swing Left", "party"] = "D"
donors.loc[donors["name_y"] == "Democratic Strategy Institute", "party"] = "D"
donors.loc[donors["name_y"] == "Senate Leadership Fund", "party"] = "R"
donors.loc[donors["name_y"] == "America First Action", "party"] = "R"
donors.loc[donors["name_y"] == "Future Forward USA", "party"] = "D"
donors.loc[donors["name_y"] == "Unite the Country", "party"] = "D"
donors.loc[donors["name_y"] == "Need to Impeach", "party"] = "D"
donors.loc[donors["name_y"] == "Tech for Campaigns", "party"] = "D"
donors.loc[donors["name_y"] == "Georgia Honor", "party"] = "D"
donors.loc[donors["name_y"] == "The Georgia Way", "party"] = "D"
donors.loc[donors["name_y"] == "Citizens for Free Enterprise", "party"] = "R"
donors.loc[donors["name_y"] == "Tom Steyer PAC", "party"] = "D"
donors.loc[donors["name_y"] == "GOPAC", "party"] = "R"
donors.loc[donors["name_y"] == "LMG PAC", "party"] = "D"
donors.loc[donors["name_y"] == "Democratic Majority for Israel", "party"] = "D"
donors.loc[donors["name_y"] == "Plains PAC", "party"] = "R"
donors.loc[donors["name_y"] == "Future Now Fund PAC", "party"] = "D"
donors.loc[donors["name_y"] == "House Freedom Action", "party"] = "R"
donors.loc[donors["name_y"] == "Way to Lead PAC", "party"] = "D"
donors.loc[donors["name_y"] == "Better Future Michigan Fund", "party"] = "R"
donors.loc[donors["name_y"] == "Digidems PAC", "party"] = "D"
donors.loc[donors["name_y"] == "Justice & Public Safety", "party"] = "D"
donors.loc[donors["name_y"] == "Casa in Action PAC", "party"] = "D"
donors.loc[donors["name_y"] == "Conservative Outsider PAC", "party"] = "R"
donors.loc[donors["name_y"] == "Save America Fund", "party"] = "D"
donors.loc[donors["name_y"] == "People Standing Strong", "party"] = "D"
donors.loc[donors["name_y"] == "State Government Citizens' Campaign", "party"] = "D"
donors.loc[donors["name_y"] == "Mind the Gap", "party"] = "D"
donors.loc[donors["name_y"] == "Elect Democratic Women", "party"] = "D"
donors.loc[donors["name_y"] == "Everyday People PAC", "party"] = "D"
donors.loc[donors["name_y"] == "For Our Families PAC", "party"] = "D"
donors.loc[donors["name_y"] == "Save the US Senate PAC", "party"] = "R"
donors.loc[donors["name_y"] == "One Vote at a Time", "party"] = "D"
donors.loc[donors["name_y"] == "Humanity Forward Fund", "party"] = "D"
donors.loc[donors["name_y"] == "One Vote at a Time", "party"] = "D"
donors.loc[donors["name_y"] == "American Patriots PAC", "party"] = "R"
donors.loc[donors["name_y"] == "Virginia Plus PAC", "party"] = "D"
donors.loc[donors["name_y"] == "Valor America", "party"] = "R"
donors.loc[donors["name_y"] == "United We Win", "party"] = "D"
donors.loc[donors["name_y"] == "New South Super PAC", "party"] = "D"
donors.loc[donors["name_y"] == "March On PAC", "party"] = "D"
donors.loc[donors["name_y"] == "L PAC", "party"] = "D"
donors.loc[donors["name_y"] == "Louisiana Legacy PAC", "party"] = "R"
donors.loc[donors["name_y"] == "Our Future United", "party"] = "D"
donors.loc[donors["name_y"] == "New American Jobs Fund", "party"] = "D"
donors.loc[donors["name_y"] == "Patriots of America PAC", "party"] = "R"
donors.loc[donors["name_y"] == "Sister District Project", "party"] = "D"
donors.loc[donors["name_y"] == "Abolitionists PAC", "party"] = "D"
donors.loc[donors["name_y"] == "California Democracy Ventures Fund", "party"] = "D"
donors.loc[donors["name_y"] == "WinRed", "party"] = "R"

In [10]:
donors.groupby(["name_y", "party"])["total_donated"].sum().sort_values(ascending=False)[:10]

name_y                               party
Michael Bloomberg (D)                D        1.086629e+09
WinRed                               R        7.205965e+08
Joe Biden (D)                        D        6.818375e+08
Donald Trump (R)                     R        4.763821e+08
Republican National Cmte             R        4.361667e+08
Biden Victory Fund                   D        3.860870e+08
Trump Make America Great Again Cmte  R        3.667061e+08
Trump Victory                        R        3.418192e+08
DNC Services Corp                    D        3.291139e+08
Tom Steyer (D)                       D        3.202495e+08
Name: total_donated, dtype: float64

In [11]:
contrib_party_totals = (
    donors.groupby(['contrib_id', 'name_new', 'party'])['total_donated']
    .sum()
    .unstack()
    .fillna(0)
)

contrib_party_totals['dem_ratio'] = (
    contrib_party_totals.get('D', 0) / 
    (contrib_party_totals.get('D', 0) + contrib_party_totals.get('R', 0))
)

donors = donors.merge(
    contrib_party_totals['dem_ratio'].reset_index(),
    on=['contrib_id', 'name_new'],
    how='left'
)

donors

,contrib_id,recip_id,name_x,name_new,realcode,gender,occupation,employer,city,state,...,cmte_id,pac_short,affiliate,pac,recip_id_pac,recipcode_pac,cand_id,party_pac,recip_is_pac,dem_ratio
0,U00000037041,N00029349,"BLOOMBERG, MICHAEL R",michael r bloomberg,Z9000,M,FOUNDER,BLOOMBERG INC.,NEW YORK,NY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1.0
1,U00000036521,N00044966,"STEYER, TOM",tom steyer,Z9000,M,PRESIDENTIAL CANDIDATE,SELF-EMPLOYED,SAN FRANCISCO,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1.0
2,,N00001669,"INFORMATION REQUESTED, INFORMATION",information information requested,Y4000,,NaN,NaN,NEW YORK,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1.0
3,,C00484642,MAJORITY FORWARD,majority forward majority forward,Z9600,,NaN,NaN,WASHINGTON,DC,...,C00484642,Senate Majority PAC,Senate Majority PAC,Senate Majority PAC,C00484642,OI,None,,True,1.0
4,,C00618371,WINRED,winred winred,Z9600,,NaN,NaN,ARLINGTON,VA,...,C00618371,Trump Make America Great Again Cmte,None,Trump Make America Great Again Cmte,C00618371,RP,None,R,True,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7136231,h3001209792,N00040675,"GOULD, COLIN",colin gould,C5120,M,SOFTWARE INFRASTRUCTURE ARCHITECT,ORACLE,FOSTER CITY,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,1.0
7136232,p0001372401,C00003418,"BLEDSOE, PEARLIOUS MS",pearlious ms bledsoe,X1200,F,RETIRED,RETIRED,GRANITEVILLE,SC,...,C00003418,Republican National Cmte,Republican National Cmte,Republican National Cmte,C00003418,RP,None,R,True,0.0
7136233,r0006423692,C00003418,"CARMICHAEL, RUTH",ruth carmichael,X1200,F,RETIRED,RETIRED,KANSAS CITY,KS,...,C00003418,Republican National Cmte,Republican National Cmte,Republican National Cmte,C00003418,RP,None,R,True,0.0
7136234,r0015995400,C00003418,"BOOKATZ, DEBRA MRS",debra mrs bookatz,Y4000,F,RETIRED,MARINE CORP. COMMUNITY SERVICES,KANEOHE,HI,...,C00003418,Republican National Cmte,Republican National Cmte,Republican National Cmte,C00003418,RP,None,R,True,0.0


In [12]:
donors["level"] = donors["seat"].apply(lambda x: "Senate" if "S1" in str(x) or "S2" in str(x) else "President" if str(x) == "PRES" else "House" if type(x) == str and len(str(x)) == 4 else "Unknown")
donors["level"].value_counts()

level
Unknown      3447415
Senate       1462832
President    1380460
House         845529
Name: count, dtype: int64

In [13]:
def read_and_process_donors_csv(year):
    
    # define files
    pred_csv = f"./output/donors_state{year}_pred_lastname.csv"
    donors_csv = f"./data/CampaignFin{year}/donors_recip{year}.csv"
    cands_csv = f"./data/CampaignFin{year}/cands{year}.txt"
    cmtes_csv = f"./data/CampaignFin{year}/cmtes{year}.txt"
    
    # read donors csv
    donors = pd.read_csv(donors_csv)
    
    # read cands csv
    cands_lf = (
        pl.scan_csv(
            cands_csv,
            separator=',', 
            quote_char='|', 
            encoding='utf8-lossy', 
            has_header=False,
            new_columns=['dummy1', 'id', 'recip_id', 'name', 'party', 
                        'seat', 'seat_current', 'ran_general', 'ran_ever', 'type', 
                        'recipcode', 'nopacs'],
            ignore_errors=True
        )
        .select(['id', 'recip_id', 'name', 'party', 'seat', 'seat_current', 
                 'ran_general', 'ran_ever', 'type', 'recipcode'])
    )

    cands = cands_lf.collect()
    cands = cands.to_pandas()
    cands = cands.drop_duplicates(subset=['recip_id', 'name', 'party', 'seat'], keep='last')
    
    # map sectors
    donors["sector"] = donors["realcode"].apply(lambda x: str(x).upper()[0]).map(mapping)
    
    # merge the three csvs
    preds = pd.read_csv(pred_csv)
    donors = donors.merge(preds[['contrib_id', 'ethnic']], on='contrib_id', how='left')
    donors = donors.merge(cands, on='recip_id', how='left')
    
    # merge pacs and committees in as recipients
    donors = donors.merge(cmtes[
        ["cmte_id", "pac_short", "affiliate", "pac", "recip_id", "recipcode", "cand_id", "party"]], 
        left_on='recip_id', right_on="cmte_id", how='left', suffixes=[None, "_pac"]
        ).drop_duplicates(subset=['contrib_id', 'recip_id', 'cmte_id'], keep='last')

    donors.loc[donors["pac"].notna(), "name_y"] = donors.loc[donors["pac"].notna(), "pac"]
    donors.loc[donors["pac_short"].notna(), "name_y"] = donors.loc[donors["pac_short"].notna(), "pac_short"]

    donors["recip_is_pac"] = False
    donors.loc[donors["pac_short"].notna(), "recip_is_pac"] = True

    donors.loc[donors["pac_short"].notna(), "party"] = donors.loc[donors["pac_short"].notna(), "party_pac"]
    
    # map party affiliations
    donors.loc[donors["name_y"] == "Stop Republicans PAC", "party"] = "D"
    donors.loc[donors["name_y"] == "Senate Majority PAC", "party"] = "D"
    donors.loc[donors["name_y"] == "314 Action Fund", "party"] = "D"
    donors.loc[donors["name_y"] == "Progressive Takeover", "party"] = "D"
    donors.loc[donors["name_y"] == "Black Americans to Re-Elect the President", "party"] = "R"
    donors.loc[donors["name_y"] == "National Victory Action Fund", "party"] = "R"
    donors.loc[donors["name_y"] == "MeidasTouch", "party"] = "D"
    donors.loc[donors["name_y"] == "Swing Left", "party"] = "D"
    donors.loc[donors["name_y"] == "Democratic Strategy Institute", "party"] = "D"
    donors.loc[donors["name_y"] == "Senate Leadership Fund", "party"] = "R"
    donors.loc[donors["name_y"] == "America First Action", "party"] = "R"
    donors.loc[donors["name_y"] == "Future Forward USA", "party"] = "D"
    donors.loc[donors["name_y"] == "Unite the Country", "party"] = "D"
    donors.loc[donors["name_y"] == "Need to Impeach", "party"] = "D"
    donors.loc[donors["name_y"] == "Tech for Campaigns", "party"] = "D"
    donors.loc[donors["name_y"] == "Georgia Honor", "party"] = "D"
    donors.loc[donors["name_y"] == "The Georgia Way", "party"] = "D"
    donors.loc[donors["name_y"] == "Citizens for Free Enterprise", "party"] = "R"
    donors.loc[donors["name_y"] == "Tom Steyer PAC", "party"] = "D"
    donors.loc[donors["name_y"] == "GOPAC", "party"] = "R"
    donors.loc[donors["name_y"] == "LMG PAC", "party"] = "D"
    donors.loc[donors["name_y"] == "Democratic Majority for Israel", "party"] = "D"
    donors.loc[donors["name_y"] == "Plains PAC", "party"] = "R"
    donors.loc[donors["name_y"] == "Future Now Fund PAC", "party"] = "D"
    donors.loc[donors["name_y"] == "House Freedom Action", "party"] = "R"
    donors.loc[donors["name_y"] == "Way to Lead PAC", "party"] = "D"
    donors.loc[donors["name_y"] == "Better Future Michigan Fund", "party"] = "R"
    donors.loc[donors["name_y"] == "Digidems PAC", "party"] = "D"
    donors.loc[donors["name_y"] == "Justice & Public Safety", "party"] = "D"
    donors.loc[donors["name_y"] == "Casa in Action PAC", "party"] = "D"
    donors.loc[donors["name_y"] == "Conservative Outsider PAC", "party"] = "R"
    donors.loc[donors["name_y"] == "Save America Fund", "party"] = "D"
    donors.loc[donors["name_y"] == "People Standing Strong", "party"] = "D"
    donors.loc[donors["name_y"] == "State Government Citizens' Campaign", "party"] = "D"
    donors.loc[donors["name_y"] == "Mind the Gap", "party"] = "D"
    donors.loc[donors["name_y"] == "Elect Democratic Women", "party"] = "D"
    donors.loc[donors["name_y"] == "Everyday People PAC", "party"] = "D"
    donors.loc[donors["name_y"] == "For Our Families PAC", "party"] = "D"
    donors.loc[donors["name_y"] == "Save the US Senate PAC", "party"] = "R"
    donors.loc[donors["name_y"] == "One Vote at a Time", "party"] = "D"
    donors.loc[donors["name_y"] == "Humanity Forward Fund", "party"] = "D"
    donors.loc[donors["name_y"] == "One Vote at a Time", "party"] = "D"
    donors.loc[donors["name_y"] == "American Patriots PAC", "party"] = "R"
    donors.loc[donors["name_y"] == "Virginia Plus PAC", "party"] = "D"
    donors.loc[donors["name_y"] == "Valor America", "party"] = "R"
    donors.loc[donors["name_y"] == "United We Win", "party"] = "D"
    donors.loc[donors["name_y"] == "New South Super PAC", "party"] = "D"
    donors.loc[donors["name_y"] == "March On PAC", "party"] = "D"
    donors.loc[donors["name_y"] == "L PAC", "party"] = "D"
    donors.loc[donors["name_y"] == "Louisiana Legacy PAC", "party"] = "R"
    donors.loc[donors["name_y"] == "Our Future United", "party"] = "D"
    donors.loc[donors["name_y"] == "New American Jobs Fund", "party"] = "D"
    donors.loc[donors["name_y"] == "Patriots of America PAC", "party"] = "R"
    donors.loc[donors["name_y"] == "Sister District Project", "party"] = "D"
    donors.loc[donors["name_y"] == "Abolitionists PAC", "party"] = "D"
    donors.loc[donors["name_y"] == "California Democracy Ventures Fund", "party"] = "D"
    donors.loc[donors["name_y"] == "WinRed", "party"] = "R"

    # calculate dem ratio
    contrib_party_totals = (
        donors.groupby(['contrib_id', 'name_new', 'party'])['total_donated']
        .sum()
        .unstack()
        .fillna(0)
    )

    contrib_party_totals['dem_ratio'] = (
        contrib_party_totals.get('D', 0) / 
        (contrib_party_totals.get('D', 0) + contrib_party_totals.get('R', 0))
    )

    donors = donors.merge(
        contrib_party_totals['dem_ratio'].reset_index(),
        on=['contrib_id', 'name_new'],
        how='left'
    )
    
    # map levels
    donors["level"] = donors["seat"].apply(lambda x: "Senate" if "S1" in str(x) or "S2" in str(x) else 
                                           "President" if str(x) == "PRES" else 
                                           "House" if type(x) == str and len(str(x)) == 4 else 
                                           "Unknown")
    
    # add electoral cycle    
    donors["cycle"] = f"20{year}"

    return donors

In [14]:
all_donors = donors
all_donors["cycle"] = f"20{year}"

for year in tqdm(list(filter(lambda x: x != year, ["00", "02", "04", "06", "08", 10, 12, 14, 16, 18, 20, 22]))):
    
    donors = read_and_process_donors_csv(year)
    all_donors = pd.concat([all_donors, donors], ignore_index=True)
    
all_donors["cycle"].value_counts()

100%|██████████| 11/11 [07:31<00:00, 41.01s/it]


cycle
2020    7136236
2022    5979614
2016    3883250
2018    3147701
2012    2355208
2008    2255971
2004    1763169
2010    1358297
2014    1257539
2006    1230286
2000    1207704
2002     974427
Name: count, dtype: int64

In [15]:
all_donors.to_csv("./output/donors_with_recip_pacs_all_years.csv", index=False)
all_donors

,contrib_id,recip_id,name_x,name_new,realcode,gender,occupation,employer,city,state,...,affiliate,pac,recip_id_pac,recipcode_pac,cand_id,party_pac,recip_is_pac,dem_ratio,level,cycle
0,U00000037041,N00029349,"BLOOMBERG, MICHAEL R",michael r bloomberg,Z9000,M,FOUNDER,BLOOMBERG INC.,NEW YORK,NY,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1.0,President,2020
1,U00000036521,N00044966,"STEYER, TOM",tom steyer,Z9000,M,PRESIDENTIAL CANDIDATE,SELF-EMPLOYED,SAN FRANCISCO,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1.0,President,2020
2,,N00001669,"INFORMATION REQUESTED, INFORMATION",information information requested,Y4000,,NaN,NaN,NEW YORK,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,False,1.0,President,2020
3,,C00484642,MAJORITY FORWARD,majority forward majority forward,Z9600,,NaN,NaN,WASHINGTON,DC,...,Senate Majority PAC,Senate Majority PAC,C00484642,OI,None,,True,1.0,Unknown,2020
4,,C00618371,WINRED,winred winred,Z9600,,NaN,NaN,ARLINGTON,VA,...,None,Trump Make America Great Again Cmte,C00618371,RP,None,R,True,0.0,Unknown,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32549397,r0014337648,C00401224,"FISKE, MARGARET",margaret fiske,Z9500,F,NaN,NaN,SAN GABRIEL,CA,...,None,ActBlue,C00401224,PI,None,None,True,NaN,Unknown,2022
32549398,m0001511409,C00401224,"POLLNOW, RITA",rita pollnow,Z9500,F,NaN,NaN,DAVIS,CA,...,None,ActBlue,C00401224,PI,None,None,True,1.0,Unknown,2022
32549399,r0007056936,C00003418,"MILLER, GLENN MR JR",glenn mr jr miller,X1200,M,RETIRED,RETIRED,OMAHA,NE,...,Republican National Cmte,Republican National Cmte,C00003418,RP,None,R,True,0.0,Unknown,2022
32549400,q0001809165,C00401224,"CAMOBELL, JEANINE",jeanine camobell,Z9500,F,NaN,NaN,PLEASANT VALLEY,NY,...,None,ActBlue,C00401224,PI,None,None,True,1.0,Unknown,2022
